0517结果

Single Scale: based on lev 3.

Multi Scale: sample and centered on lev 3, expand to lev 4.

先确定single 能跑，multi能跑。再multi同时产生大量lev3&lev4的图，跑multi，再跑single.

最后比较两者在lev3的heatmap.

最后的最后，roc, froc之类

# 1. Single Scale, Train Step

## 1.1. Random sample center point

## 1.2. Generate patches

In [0]:
# Expand a 299*299 patch around the center tissue
ipatch = read_slide(slide, 
                   x=int(center[0]-149)*down_factor[ilevel], #16 for lev4, 8 for lev3 
                   y=int(center[1]-149)*down_factor[ilevel], 
                   level=ilevel, 
                   width=299, 
                   height=299)

# Check whether the 128*128 center pixels(lev3) have tumor
ipixels = read_slide(tumor_mask,
                     x = int(center[0]-63)*down_factor[ilevel], 
                     y = int(center[1]-63)*down_factor[ilevel],
                     level = ilevel,
                     width = 128,
                     height = 128)[:,:,0]

# 2. Single scale, testing step

In [0]:
#stride on the center pixcel, center[0] = 0.1w+128i, center[1] 0.1h+128j
test_center = []
test_patches = []
stride = 128
bound = 149 # (299-1)//2
width = slide.level_dimensions[ilevel][0]
height = slide.level_dimensions[ilevel][1]
nx = (int(width*0.9)-int(width*0.1))//stride + 1
ny = (int(height*0.9)-int(height*0.1))//stride + 1
for i in range(nx):
  for j in range(ny):
    patch_ij = read_slide(slide,
                          x = int(0.1*width+stride*i-bound)*down_factor[ilevel],
                          y = int(0.1*height+stride*j-bound)*down_factor[ilevel],
                          level = ilevel,
                          width = input_size,
                          height = input_size)
    test_patches.append(patch_ij)
    test_center.append((int(0.1*width+stride*i-63), int(0.1*height+stride*j-63)))
    


after testing, return predicted prob with coordinate from test_center

# 3. Multi-scale, Training


## 3.1. Random sample center point on lev 3

## 3.2. Generate patches on both lev3&lev4

In [0]:
# Expand a 299*299 patch around the center tissue
ipatch3 = read_slide(slide, 
                   x=int(center[0]-149)*down_factor[ilevel], #16 for lev4, 8 for lev3 
                   y=int(center[1]-149)*down_factor[ilevel], 
                   level=ilevel, 
                   width=299, 
                   height=299)

# Check whether the 128*128 center pixels(lev3) have tumor
ipixels = read_slide(tumor_mask,
                     x = int(center[0]-63)*down_factor[ilevel], 
                     y = int(center[1]-63)*down_factor[ilevel],
                     level = ilevel,
                     width = 128,
                     height = 128)[:,:,0]

确定要了，再lev4

In [0]:
ipatch4 = read_slide(slide, 
                   x=int(center[0]-149)*8, #16 for lev4, 8 for lev3 
                   y=int(center[1]-149)*8, 
                   level=4, 
                   width=299, 
                   height=299)

# 4. Multi scale, Test

In [0]:
#stride on the center pixcel, center[0] = 0.1w+128i, center[1] 0.1h+128j
test_center = []
test_patches3 = []
test_patches4 = []
stride = 128
bound = 149 # (299-1)//2
width = slide.level_dimensions[ilevel][0]
height = slide.level_dimensions[ilevel][1]
nx = (int(width*0.9)-int(width*0.1))//stride + 1
ny = (int(height*0.9)-int(height*0.1))//stride + 1
for i in range(nx):
  for j in range(ny):
    ipatch_3 = read_slide(slide,
                          x = int(0.1*width+stride*i-bound)*down_factor[ilevel],
                          y = int(0.1*height+stride*j-bound)*down_factor[ilevel],
                          level = ilevel,
                          width = input_size,
                          height = input_size)
    ipatch_4 = read_slide(slide,
                          x = int(0.1*width+stride*i-bound)*down_factor[ilevel],
                          y = int(0.1*height+stride*j-bound)*down_factor[ilevel],
                          level = 4,
                          width = input_size,
                          height = input_size)
    test_patches3.append(ipatch_3)
    test_patches4.append(ipatch_4)
    test_center.append((int(0.1*width+stride*i-63), int(0.1*height+stride*j-63)))
    


只slide on lev3. lev 4是同比例放大。